<h1>Culture Match Data Cleaning</h1>
Now that we have scraped our required information from Glassdoor, let's clean it up so that it will be useful in our matching model.

In [209]:
#import required libraries
import pandas as pd

#import datasets
df_jobs = pd.read_csv("scraped_df.csv")
df_companies = pd.read_csv("fortune_500.csv")

Let's look at the columns to see what we have imported for both databases.

In [210]:
df_jobs.head()

,Job Title,Salary Estimate,Job Description,Rating,Company Name,Location,Headquarters,Size,Founded,Type of ownership,...,Revenue,Competitors,Word count,civic,family,industrial,inspired,market,popular,sustainable
0,CDL-A Truck Driver - Walmart Transportation,-1,"LOCATION HURRICANE, UT\nCAREER AREA Drivers & ...",3.3,Walmart\n3.3,"Washington, UT","Bentonville, AR",1001 to 5000 employees,1962,Company - Public,...,$10+ billion (USD),"Target, Costco Wholesale, Amazon",612,2,6,5,8,10,5,9
1,Automotive Technician,-1,Automotive Technician\n\nYoull discover more t...,3.3,Walmart\n3.3,"Albert Lea, MN","Bentonville, AR",1001 to 5000 employees,1962,Company - Public,...,$10+ billion (USD),"Target, Costco Wholesale, Amazon",195,1,2,2,6,6,1,5
2,Truck Driver - Night Runs (Home Daily),-1,"LOCATION POTTSVILLE, PA\nCAREER AREA Drivers &...",3.3,Walmart\n3.3,"Shenandoah, PA","Bentonville, AR",1001 to 5000 employees,1962,Company - Public,...,$10+ billion (USD),"Target, Costco Wholesale, Amazon",627,2,6,5,8,10,5,9
3,Warehouse Worker,-1,"As a Freight Handler at Walmart Supply Chain, ...",3.3,Walmart\n3.3,"Pauls Valley, OK","Bentonville, AR",1001 to 5000 employees,1962,Company - Public,...,$10+ billion (USD),"Target, Costco Wholesale, Amazon",262,5,0,4,6,5,0,11
4,Freight Handler,-1,"As a Freight Handler at Walmart Supply Chain, ...",3.3,Walmart\n3.3,"Pauls Valley, OK","Bentonville, AR",1001 to 5000 employees,1962,Company - Public,...,$10+ billion (USD),"Target, Costco Wholesale, Amazon",262,5,0,4,6,5,0,11


In [211]:
df_companies.head()

,Rank,Title,Website,Employees,Sector,Industry,Hqlocation,Hqaddr,Hqcity,Hqstate,...,Revenues,Revchange,Profits,Prftchange,Assets,Totshequity,hiring,hiring_freeze,layoffs,Offers rescinded
0,94,3M,http://www.3m.com,91584,Industrials,Miscellaneous,"St. Paul, MN",3M Center,St. Paul,MN,...,30109,-0.5,5050.0,4.5,32906,10298.0,1.0,1.0,0.0,0.0
1,135,Abbott Laboratories,http://www.abbott.com,75000,Health Care,Medical Products and Equipment,"Abbott Park, IL",100 Abbott Park Rd.,Abbott Park,IL,...,20853,0.9,1400.0,-68.3,52666,20538.0,1.0,1.0,0.0,0.0
2,111,AbbVie,http://www.abbvie.com,30000,Health Care,Pharmaceuticals,"North Chicago, IL",1 N. Waukegan Rd.,North Chicago,IL,...,25638,12.2,5953.0,15.7,66099,4636.0,1.0,1.0,0.0,0.0
3,500,ABM Industries,http://www.abm.com,110000,Business Services,Diversified Outsourcing Services,"New York, NY",1 Liberty Plaza,New York,NY,...,5145,-2.8,57.2,-25,2281,974.0,1.0,NaN,NaN,NaN
4,406,Activision Blizzard,http://www.activisionblizzard.com,9500,Technology,Computer Software,"Santa Monica, CA",3100 Ocean Park Blvd.,Santa Monica,CA,...,6608,41.7,966.0,8.3,17452,9119.0,1.0,NaN,NaN,NaN


Let's remove the columns that we won't need

In [212]:
df_jobs = df_jobs.drop(columns = ['Job Title','Job Description','Salary Estimate','Rating',
                   'Headquarters','Industry','Sector','Size','Location','Founded',
                   'Type of ownership','Revenue','Competitors'])

In [213]:
df_companies = df_companies.drop(columns = ['Rank', 'Website', 'Hqlocation', 'Hqaddr', 'Hqcity', 
                           'Hqzip', 'Hqtel', 'Ceo','Ceo-title', 'Address', 'Ticker', 
                           'Fullname', 'Revenues', 'Revchange','Profits', 'Prftchange', 
                           'Assets', 'Totshequity', 'hiring','hiring_freeze', 
                           'layoffs', 'Offers rescinded'])

Let's collapse the df of the job descriptions

Now we should try to join the two tables.
We will do this with the name.
While it looks like the Company name is clean in df_companies, we will use that as the base to match with. Since Company name in df_jobs is full of regex and numbers and letters, we will split apart the name and then match the first word to the name in df_companies.

In [214]:
#set title of company data base to first word and lowercase
df_companies['updated_title'] = df_companies.apply(lambda x : x['Title'].rsplit()[0].lower(),axis=1)
df_companies = df_companies.drop(columns =['Title'])

#set title of job data base to lowercase and remove extra characters
df_jobs['company_name'] = df_jobs.apply(lambda x:x['Company Name'].rsplit("\n")[0].lower(),axis=1)
df_jobs = df_jobs.drop(columns = ['Company Name'])

for i in range(0,len(df_jobs)):
    for u in range(0,len(df_companies)):
        if df_jobs['company_name'][i] == df_companies['updated_title'][u]:
            df_jobs.loc[i,'employees'] = df_companies['Employees'][u]
            df_jobs.loc[i,'sector'] = df_companies['Sector'][u]
            df_jobs.loc[i,'industry'] = df_companies['Industry'][u]
            df_jobs.loc[i,'location'] = df_companies['Hqstate'][u]

Now we can drop the non matches. The rows that weren't matched will have a na we can just drop those rows.

In [215]:
df_jobs=df_jobs.dropna()

In [216]:
df_jobs

,Word count,civic,family,industrial,inspired,market,popular,sustainable,company_name,employees,sector,industry,location
0,612,2,6,5,8,10,5,9,walmart,2300000.0,Retailing,General Merchandisers,AR
1,195,1,2,2,6,6,1,5,walmart,2300000.0,Retailing,General Merchandisers,AR
2,627,2,6,5,8,10,5,9,walmart,2300000.0,Retailing,General Merchandisers,AR
3,262,5,0,4,6,5,0,11,walmart,2300000.0,Retailing,General Merchandisers,AR
4,262,5,0,4,6,5,0,11,walmart,2300000.0,Retailing,General Merchandisers,AR
...,...,...,...,...,...,...,...,...,...,...,...,...,...
5864,389,6,4,19,3,11,0,6,abm,110000.0,Business Services,Diversified Outsourcing Services,NY
5865,103,3,0,2,0,1,0,8,abm,110000.0,Business Services,Diversified Outsourcing Services,NY
5870,79,2,0,2,0,2,0,4,abm,110000.0,Business Services,Diversified Outsourcing Services,NY
5871,223,5,3,8,1,2,0,5,abm,110000.0,Business Services,Diversified Outsourcing Services,NY


At this point we have cleaned the data well enough to be used in the rest of our data pipeline.

In [217]:
df_jobs.to_csv(r'cc_cleaned_2.csv', index = False)